In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC 
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.algorithms import QSVC
from sklearn.decomposition import PCA


In [ ]:
df = pd.read_csv('stunting.csv')
df.info()

In [ ]:
df = df.drop_duplicates()

In [ ]:
print(df)

In [ ]:
df = df.drop('Wasting', axis=1)

In [ ]:
df.info()

In [ ]:
label_encoder = LabelEncoder()

# Loop through each column and apply LabelEncoder to object columns
for column in df.select_dtypes(include=['object']).columns:
    df[column] = label_encoder.fit_transform(df[column])

In [ ]:
correlation_matrix = df.corr()

# Set the figure size
plt.figure(figsize=(8, 6))

# Create the heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)

# Add title
plt.title('Correlation Matrix')

# Show the plot
plt.show()

In [ ]:
from sklearn.utils import resample

target_per_class = 100   # karena 1,000 ÷ 4 kelas

frames = []
for label, subset in df.groupby('Stunting'):
    if len(subset) >= target_per_class:
        frames.append(subset.sample(n=target_per_class, random_state=42))
    else:
        frames.append(
            resample(subset, replace=True, n_samples=target_per_class, random_state=42)
        )

df_reduced = pd.concat(frames).sample(frac=1, random_state=42).reset_index(drop=True)
print(df_reduced['Stunting'].value_counts()) 


In [ ]:
# df_balanced berisi data final yang sudah seimbang
file_path = "stunting_dataset.csv"   # nama berkas yang diinginkan

# index=False agar kolom index pandas tidak ikut disimpan
df_reduced.to_csv(file_path, index=False)

print(f"File tersimpan di: {file_path}")


In [ ]:
X = df_reduced.drop('Stunting', axis=1)
y = df_reduced['Stunting']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
pca = PCA(n_components=0.9, svd_solver='full')   # jaga ≥95 % varians
X_train_red = pca.fit_transform(X_train_scaled)
X_test_red  = pca.transform(X_test_scaled)
num_qubits  = X_train.shape[1]

In [ ]:
# Angle Encoding berbasis FeatureMap
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap

num_qubit = X_train.shape[1]
feature_map = ZZFeatureMap(feature_dimension=num_qubit, reps=1, entanglement="linear")
feature_map.decompose().draw(output="mpl", style="clifford", fold=100)

In [ ]:
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

sampler  = Sampler()                       # simulator presisi penuh
fidelity = ComputeUncompute(sampler=sampler)
quantum_kernel = FidelityQuantumKernel(
    fidelity=fidelity,
    feature_map=feature_map
)

In [35]:
model = SVC(kernel=quantum_kernel.evaluate)
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
acc = accuracy_score(y_test, y_pred_test)
print(f"Akurasi QSVM test : {acc:.3f}")


KeyboardInterrupt: 

In [ ]:
print('akurasi train')
print("\n=== Classification Report ===")
print(classification_report(y_train, y_pred_train))
print('akurasi test')
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred_test))